# This code shoud run once a day to generate fact and dim tables

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Loading Dim_Battalion

In [3]:
dim_battalion = pd.read_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/dimension_tables/dim_battalion_fire_incidents.parquet")
dim_battalion.head(10)

,battalion,station_area,battalion_id
0,B06,07,1
1,B06,11,2
2,B01,02,3
3,B05,12,4
4,B01,01,5
5,B03,13,6
6,B09,43,7
7,B03,01,8
8,B02,03,9
9,B08,22,10


# Loading Dim_District

In [4]:
dim_district = pd.read_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/dimension_tables/dim_district_fire_incidents.parquet")
dim_district.head(10)

,neighborhood_district,supervisor_district,district_id
0,Mission,9,1
1,Chinatown,3,2
2,Inner Sunset,8,3
3,Nob Hill,3,4
4,Financial District/South Beach,6,5
5,Visitacion Valley,10,6
6,Tenderloin,5,7
7,South of Market,6,8
8,Western Addition,2,9
9,Sunset/Parkside,4,10


# Loading Dim_Time

In [5]:
dim_time = pd.read_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/dimension_tables/dim_time_fire_incidents.parquet")
dim_time.head(10)

,incident_date,year,month,day,day_of_week,date_id
0,2017-02-12,2017,2,12,6,1
1,2017-02-13,2017,2,13,0,2
2,2017-02-14,2017,2,14,1,3
3,2017-02-15,2017,2,15,2,4
4,2017-02-16,2017,2,16,3,5
5,2017-02-17,2017,2,17,4,6
6,2017-02-18,2017,2,18,5,7
7,2017-02-19,2017,2,19,6,8
8,2017-02-20,2017,2,20,0,9
9,2017-02-21,2017,2,21,1,10


# Loading Fact_Table

In [13]:
fact_fire_incidents = pd.read_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/fact_table/fact_fire_incidents.parquet")
fact_fire_incidents.head(10)

,incident_number,date_id,district_id,battalion_id,primary_situation,station_area,fire_fatalities,civilian_fatalities,estimated_property_loss
0,17019056,1,1,1,733 Smoke detector activation due to malfunction,07,0,0,0.0
1,17019056,1,29,1,733 Smoke detector activation due to malfunction,07,0,0,0.0
2,17019056,1,45,1,733 Smoke detector activation due to malfunction,07,0,0,0.0
3,17019056,1,51,1,733 Smoke detector activation due to malfunction,07,0,0,0.0
4,17019056,1,1,2,733 Smoke detector activation due to malfunction,07,0,0,0.0
5,17019056,1,29,2,733 Smoke detector activation due to malfunction,07,0,0,0.0
6,17019056,1,45,2,733 Smoke detector activation due to malfunction,07,0,0,0.0
7,17019056,1,51,2,733 Smoke detector activation due to malfunction,07,0,0,0.0
8,17019056,1,1,16,733 Smoke detector activation due to malfunction,07,0,0,0.0
9,17019056,1,29,16,733 Smoke detector activation due to malfunction,07,0,0,0.0


## This report provides a summary of fire incidents by date, district, and situation, showing how many incidents occurred, the total property loss, and the fatalities associated with each type of incident in each location and time period.

In [21]:
report_agg_incidents = (
    fact_fire_incidents
    .merge(dim_time, on='date_id', how='left')
    .merge(dim_battalion, on='battalion_id', how='left')
    .merge(dim_district, on='district_id', how='left')
    .groupby(["incident_date", "neighborhood_district", "primary_situation"])
    .agg({
        'incident_number': 'count',
        'estimated_property_loss': 'sum',
        'fire_fatalities': 'sum',
        'civilian_fatalities': 'sum'
    })
    .reset_index()
    .rename(columns={'incident_number': 'incident_count'})
)
report_agg_incidents

,incident_date,neighborhood_district,primary_situation,incident_count,estimated_property_loss,fire_fatalities,civilian_fatalities
0,2017-02-12,Mission,733 Smoke detector activation due to malfunction,36,0.0,0,0
1,2017-02-13,Bayview Hunters Point,323 Motor vehicle/pedestrian accident (MV Ped),8,0.0,0,0
2,2017-02-13,Bayview Hunters Point,"510 Person in distress, other",8,0.0,0,0
3,2017-02-13,Bayview Hunters Point,"520 Water problem, other",8,0.0,0,0
4,2017-02-13,Bayview Hunters Point,"700 False alarm or false call, other",16,0.0,0,0
...,...,...,...,...,...,...,...
870,2017-02-25,South of Market,353 Removal of victim(s) from stalled elevator,8,0.0,0,0
871,2017-02-25,South of Market,522 Water or steam leak,8,0.0,0,0
872,2017-02-25,South of Market,554 Assist invalid,8,0.0,0,0
873,2017-02-25,Tenderloin,322 Motor vehicle accident with injuries,21,0.0,0,0


In [ ]:
report_agg_incidents.to_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/reports/agg_incidents.parquet")

## This report summarizes the number of incidents, property loss, and fatalities by neighborhood and month. It would allow you to track fire incidents' impact across districts over time, helping in resource allocation and identifying areas requiring more preventive measures.

In [22]:
report_agg_neighborhood = (
    fact_fire_incidents
    .merge(dim_time, on='date_id', how='left')
    .merge(dim_battalion, on='battalion_id', how='left')
    .merge(dim_district, on='district_id', how='left')
    .groupby(["year", "month", "neighborhood_district"])
    .agg({
        'incident_number': 'count',
        'estimated_property_loss': 'sum',
        'fire_fatalities': 'sum',
        'civilian_fatalities': 'sum'
    })
    .reset_index()
    .rename(columns={'incident_number': 'incident_count'})
)

report_agg_neighborhood.head(10)

,year,month,neighborhood_district,incident_count,estimated_property_loss,fire_fatalities,civilian_fatalities
0,2017,2,Bayview Hunters Point,504,9700000.0,0,0
1,2017,2,Bernal Heights,171,0.0,0,0
2,2017,2,Castro/Upper Market,164,600.0,0,0
3,2017,2,Chinatown,119,0.0,0,0
4,2017,2,Excelsior,126,0.0,0,0
5,2017,2,Financial District/South Beach,1222,84680.0,0,0
6,2017,2,Glen Park,54,9.0,0,0
7,2017,2,Golden Gate Park,171,0.0,0,0
8,2017,2,Haight Ashbury,108,0.0,0,0
9,2017,2,Hayes Valley,564,798000.0,0,0


In [23]:
report_agg_neighborhood.to_parquet("/Users/reinaldoduzanskijunior/Documents/teste/fire_incidents/data_models/reports/report_agg_neighborhood.parquet")